# EmoBert Evaluation 


This notebook shows a quick evaluation of the EmoBert model trained in `Build-emotions-BERT-model-final.ipynb`.

In [2]:
import pickle
from limbic.emotion.models.bert_limbic_model.bert_limbic_model import BertLimbicModel


In [3]:
bert_path = '../data/distill_bert'
model_path = '../data/models/emo_bert_model_2021-01-10.bin'
 
bert_model = BertLimbicModel(model_path, bert_path)

Some weights of the model checkpoint at ../data/distill_bert were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'dist

In [28]:
bert_model.get_sentence_emotions('I have a lot of joy')

[EmotionValue(category='sadness', value=0.0003411198),
 EmotionValue(category='joy', value=0.9990711),
 EmotionValue(category='fear', value=0.0003754736),
 EmotionValue(category='anger', value=0.00021218295)]

In [24]:
import pandas as pd
import tensorflow as tf

from limbic.limbic_constants import AFFECT_INTENSITY_EMOTIONS as EMOTIONS
from limbic.emotion.models.tf_limbic_model import utils

SENTENCE_EMOTIONS_TEST_FILE = '../data/sentence_emotions_test.pickle' 
CONTINUES_TO_BINARY_THRESHOLD = 0.5


def load_data_file(file_path):
    data = pd.read_pickle(file_path)
    data_sentences = data['text']
    y_data = data[EMOTIONS].values
    y_data_labeled = utils.continuous_labels_to_binary(y_data, CONTINUES_TO_BINARY_THRESHOLD)   
    
    return data, y_data, y_data_labeled, data_sentences

test, y_test, y_test_labeled, test_sentences = load_data_file(SENTENCE_EMOTIONS_TEST_FILE)

print(f'test shape: {test.shape}')

test shape: (19085, 5)


In [25]:
from tqdm.notebook import tqdm 


y_pred_bert = []
for sentence in tqdm(test_sentences):
    prediction = bert_model.predict(sentence)
    y_pred_bert.append(prediction)

  0%|          | 0/19085 [00:00<?, ?it/s]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_pred_bert_labeled = utils.continuous_labels_to_binary(np.array([list(x) for x in y_pred_bert]), 0.2)
print(classification_report(y_test_labeled, y_pred_bert_labeled, target_names=EMOTIONS))


              precision    recall  f1-score   support

     sadness       0.14      0.54      0.22      2430
         joy       0.24      0.83      0.38      3603
        fear       0.16      0.65      0.26      2684
       anger       0.09      0.42      0.15      1783

   micro avg       0.17      0.65      0.26     10500
   macro avg       0.16      0.61      0.25     10500
weighted avg       0.17      0.65      0.27     10500
 samples avg       0.32      0.26      0.27     10500



/Users/gastonlhuillier/Personal/limbic/.python/limbic/lib/python3.7/site-packages/scikit_learn-0.21.3-py3.7-macosx-10.14-x86_64.egg/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
